In [31]:
from safe_control_gym.envs.manipulators.manipulator import BaseManipulator
import numpy as np 
import random 
from time import sleep
from safe_control_gym.envs.benchmark_env import Cost, Task
from gym import spaces
from stable_baselines3 import PPO, DDPG, A2C

In [3]:
urdf_path = "../safe_control_gym/envs/manipulators/assets/franka_panda/panda.urdf"
controlled_variable = "torque"
target_space = "joint"

controlled_joint_indices = [6]
observed_link_indices = [7]
observed_link_state_keys = ["position"]


# orientation_goal = [0.603, 0.3687, -0.3697, 0.6026]
# position_goal = [0.03692, 0, 0.973]

# position_goal = [-0.127, 0.0, 1.012] # for 1 joint 
# -0.03458691, -0.01447242,  1.118461
# position_goal = [0.0212 , -0.0288,  1.113] # 2joints

goal = [0.0212, 1.113]
# position_goal = [0.0868, -0.022,  1.014] # 2 joints
# position_goal = [0.0, 0, 1]

#TODO reuse this for future 
# goal = [{
#     "position": [None for i in range(13)]
# }]
# goal[0]["position"][observed_link_indices[0]] = position_goal

goal_type = "point"
connection = "gui"
control_method = "classical"
dimensions = 2

if dimensions == 2:
    position_goal = [0.0212, 1.113]
    
elif dimensions == 3:
    position_goal = [0.0212, 0, 1.113]
    
env = BaseManipulator(
    urdf_path,
    controlled_variable,
    control_method,
    target_space,
    controlled_joint_indices = controlled_joint_indices,
    observed_link_indices = observed_link_indices, 
    observed_link_state_keys = observed_link_state_keys,
    goal = goal,
    goal_type = goal_type,
    cost = Cost.RL_REWARD,
    connection = connection,
    dimensions = dimensions
    
)

argv[0]=
Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1 Pro
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [11]:
env.reset()
action_space = [4, 3]
# action_space = [-400]

for i in range(50):
#     action_index = env.n_joints-2
    action_index = 6
    # action_list = np.zeros(env.n_joints)
    action_list = [random.choice(action_space) for i in env.controlled_joint_indices]    
    joint_states=  env._pb_client.getJointState(env.robot,
                                            jointIndex=6,
                                )
    # action_list = [200]
    link_states =  env._pb_client.getLinkState(env.robot,
                                            linkIndex=7,
                                            computeLinkVelocity=True)[-1]

    obs,reward, done,info = env.step(action_list)
    # if i%5== 0 :
    # print(action_list)
    obs_position = [round(i,3) for i in obs[:3]]
    
    print(f"obs: {obs_position}") # link position
    # print(f"reward: {reward}")
  #   print(f"angular velocity: {obs}") # velocity
    # print(f"joint state: {joint_states}") # link position
    # print(f"link state: {link_states}") # link position
    # print(f"observation: {}")
    # print(f"position : {link_states[]}")
    sleep(0.05)

obs: [0.088, -0.0, 0.953]
obs: [0.09, 0.0, 0.955]
obs: [0.096, 0.0, 0.963]
obs: [0.103, 0.0, 0.973]
obs: [0.11, 0.0, 0.987]
obs: [0.115, 0.0, 1.004]
obs: [0.119, -0.0, 1.029]
obs: [0.117, 0.0, 1.061]
obs: [0.108, 0.0, 1.093]
obs: [0.095, 0.0, 1.119]
obs: [0.08, 0.0, 1.137]
obs: [0.071, 0.0, 1.147]
obs: [0.064, 0.0, 1.152]
obs: [0.058, -0.0, 1.151]
obs: [0.06, -0.0, 1.147]
obs: [0.067, -0.0, 1.141]
obs: [0.079, -0.0, 1.133]
obs: [0.093, -0.0, 1.121]
obs: [0.108, -0.0, 1.106]
obs: [0.125, -0.0, 1.085]
obs: [0.141, 0.0, 1.058]
obs: [0.159, 0.0, 1.055]
obs: [0.177, 0.0, 1.05]
obs: [0.193, 0.0, 1.046]
obs: [0.208, 0.0, 1.042]
obs: [0.222, 0.0, 1.038]
obs: [0.235, 0.0, 1.033]
obs: [0.247, 0.0, 1.029]
obs: [0.258, 0.0, 1.025]
obs: [0.268, 0.0, 1.022]
obs: [0.277, 0.0, 1.018]
obs: [0.285, 0.0, 1.015]
obs: [0.292, 0.0, 1.012]
obs: [0.298, 0.0, 1.009]
obs: [0.303, 0.0, 1.007]
obs: [0.306, 0.0, 1.005]
obs: [0.309, 0.0, 1.004]
obs: [0.312, 0.0, 1.003]
obs: [0.313, 0.0, 1.002]
obs: [0.313, 0.0, 1.0

In [ ]:
env.reset()

Exploring link states

In [7]:
joint_states=  env._pb_client.getJointState(env.robot,
                                            jointIndex=6,
                                )
joint_states

(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)

In [6]:
joint_states=  env._pb_client.getJointState(env.robot,
                                            jointIndex=7,
                                )

joint_states

(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)

In [ ]:
link_states =  env._pb_client.getLinkState(env.robot,
                                            linkIndex=7,
                                            computeLinkVelocity=True)


In [ ]:
link_states

In [ ]:
# only extract world positions
link_states_dict = {
    "position": link_states[4],
    "orientation": link_states[5],
    "linear_vel": link_states[6],
    "angular_vel": link_states[7]
}

In [ ]:
link_states_dict

## RL

In [4]:
from gym import spaces 
# env.observation_space = spaces.Box(-np.inf, np.inf, (14,), dtype=np.float32)

# orientation use quaternion so 4 dim 

# action_space = [spaces.Discrete(1) for i in range(env.n_joints-1)]
# action_space.append(spaces.Box(-400, 400, shape=(1,), dtype=np.float32))

# env.action_space = spaces.Tuple(
#     action_space
# )

# dim dict
dim_dict = {
    "position":3 + 3,
    "orientation" : 4+4
}
env.action_space = spaces.Box(-4.5, 4.5, (len(env.controlled_joint_indices),), dtype=np.float32)

# TODO expand to multiple state keys 
env.observation_space = spaces.Box(-np.inf, np.inf, (dim_dict[observed_link_state_keys[0]], ), dtype=np.float32)

# env.observation_space = spaces.Dict({
#       'position': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#       'orientation': spaces.Box(-np.inf, np.inf, (env.n_joints, 4), dtype=np.float32),
#       'linear_vel': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#       'angular_vel': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#     })


In [5]:
env._get_observation()

array([ 0.09856006,  0.04260633,  1.12294292,  0.0212    , -0.0288    ,
        1.113     ])

In [6]:

env.reset()
ppo_model = PPO('MlpPolicy', env, verbose=1)

# use this to stop if too many timesteps cannot converge to done 
for i in range(10):
    ppo_model.learn(total_timesteps=10000)




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 72   |
|    iterations      | 1    |
|    time_elapsed    | 28   |
|    total_timesteps | 2048 |
-----------------------------


KeyboardInterrupt: 

In [4]:

# env.reset()
ppo_model = PPO('MlpPolicy', env, verbose=1)

# use this to stop if too many timesteps cannot converge to done 
for i in range(10):
    ppo_model.learn(total_timesteps=10000)




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 76   |
|    iterations      | 1    |
|    time_elapsed    | 26   |
|    total_timesteps | 2048 |
-----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 82            |
|    iterations           | 2             |
|    time_elapsed         | 49            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00020561198 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.84         |
|    explained_variance   | -1.86e-05     |
|    learning_rate        | 0.0003        |
|    loss                 | 8.06e+06      |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.00010

In [ ]:
ppo_model.save("manipulator_100000")

In [9]:
obs = env.reset()
for i in range(3000):
    action, _state = ppo_model.predict(obs, deterministic=False)
    # print(_state)
    obs, reward, done, info = env.step(action)
    # applied_action = env._action_mapping_torque(action)
    
    if i%5==0:
        print(f"Iteration:{i}")
        print(f"action: {action}")
        print(f"goal: {goal[0]['position'][7]}")
        print(f"state: {[round(ob,3) for ob in obs]}")
        print(f"reward: {reward}\n")
    
    if done: 
        print("done")
        break
    
    # sleep(0.05)

Iteration:0
action: [-0.5811313   0.56101274]
goal: [0.0212, -0.0288, 1.113]
state: [0.088, 0.0, 1.033, 0.021, -0.029, 1.113]
reward: -175.60000715289036

Iteration:5
action: [ 0.05037165 -0.66742194]
goal: [0.0212, -0.0288, 1.113]
state: [0.088, -0.003, 1.029, 0.021, -0.029, 1.113]
reward: -176.4528595138341

Iteration:10
action: [0.11259472 0.725588  ]
goal: [0.0212, -0.0288, 1.113]
state: [0.087, 0.002, 1.027, 0.021, -0.029, 1.113]
reward: -183.29054464064538

Iteration:15
action: [-0.3309157   0.06978023]
goal: [0.0212, -0.0288, 1.113]
state: [0.088, 0.0, 1.035, 0.021, -0.029, 1.113]
reward: -173.11900618895888

Iteration:20
action: [ 0.7653686 -1.6649845]
goal: [0.0212, -0.0288, 1.113]
state: [0.084, -0.017, 1.05, 0.021, -0.029, 1.113]
reward: -138.6604439228773

Iteration:25
action: [-0.7841464   0.49291724]
goal: [0.0212, -0.0288, 1.113]
state: [0.087, -0.009, 1.038, 0.021, -0.029, 1.113]
reward: -161.35404070317745

Iteration:30
action: [ 0.35791013 -0.30590567]
goal: [0.0212, 

## Classical Control

In [1]:
from safe_control_gym.envs.manipulators.manipulator import BaseManipulator
import numpy as np 
import random 
from time import sleep
from safe_control_gym.envs.benchmark_env import Cost, Task
from gym import spaces
from stable_baselines3 import PPO, DDPG, A2C
import os 
home = "/Users/nicholasprayogo/code/rm-sc"
# cwd = os.path.join(home, "src/interpretable_ts_clustering/")
os.chdir(home)

from functools import partial
from safe_control_gym.utils.configuration import ConfigFactory

fac = ConfigFactory()
config = fac.merge()
# env_func = partial(make, config.task, output_dir=config.output_dir, **config.task_config)

pybullet build time: Mar 29 2022 22:56:14
/Users/nicholasprayogo/code/rm-sc/safe_control_gym/utils/configuration.py:72: UserWarning: No agent/task config given.
  warnings.warn("No agent/task config given.")


In [2]:
config

Munch({'tag': 'temp', 'seed': None, 'device': 'cpu', 'output_dir': 'results', 'restore': None})

In [3]:
import yaml

config.algo = "lqr"
config.task = "manipulator"
controller_yaml_path = "safe_control_gym/controllers/lqr/lqr.yaml"
env_yaml_path = "safe_control_gym/envs/manipulators/manipulator.yaml"

with open(controller_yaml_path, "r") as yamlfile:
    config.algo_config = yaml.load(yamlfile, Loader=yaml.FullLoader)
    
with open(env_yaml_path, "r") as yamlfile:
    config.task_config = yaml.load(yamlfile, Loader=yaml.FullLoader)

In [4]:
from safe_control_gym.utils.registration import make
env_func = partial(make, config.task, output_dir=config.output_dir, **config.task_config)
control_agent = make(config.algo,
                         env_func,
                         training=True,
                         checkpoint_path=os.path.join(config.output_dir, "model_latest_manippulator.pt"),
                         output_dir=config.output_dir,
                         device=config.device,
                         seed=config.seed,
                         **config.algo_config)

@1=vertcat(cos(theta), sin(theta)), (mac(mac((0.5*(@1-Xr)'),Q,zeros(1x2)),(@1-Xr),0)+(((0.5*(U-Ur))*R)*(U-Ur)))argv[0]=
Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1 Pro
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started



RuntimeError: Error in Function::Function for 'fc' [MXFunction] at .../casadi/core/function.cpp:235:
.../casadi/core/function_internal.cpp:145: Error calling MXFunction::init for 'fc':
.../casadi/core/x_function.hpp:287: For fc: Xfunction input arguments must be purely symbolic.
Argument 0(x) is not symbolic.

In [ ]:
env_func

functools.partial(<function make at 0x7fb1e07111f0>, 'lqr_manipulator', output_dir='results', test='test')

In [17]:
from safe_control_gym.utils.registration import register

register(id="manipulator",
         entry_point="safe_control_gym.envs.manipulators.manipulator:BaseManipulator",
         config_entry_point="safe_control_gym.envs.manipulators:manipulator.yaml")


In [21]:
!python3 experiments.py

zsh: Can't add module parameter `commands': parameter already exists
CITATION.cff     examples         pyproject.toml   setup.py
LICENSE          experiments      results          walkthroughs
README.md        figures          safe_control_gym


In [33]:
import casadi as cs

g = 9.8
dt = 0.005 
mc = 0.5
ma = 0.5
# TODO express in terms of actual length (distance b.w 2 joints)
l = 1
I = 0 # moment of inertia from gears and motors

# must match dimension of X_GOAL and U_GOAL
nx = 2
nu = 1 

# x = cs.MX.sym('x')

x_x = cs.MX.sym('x_x')
# x_y = cs.MX.sym('x_y')
x_z = cs.MX.sym('x_z')
theta = cs.MX.sym('theta')
U = cs.MX.sym('U') # torque
theta_dot_dot = cs.MX.sym("tdotdot")
Theta_dot_dot = cs.vertcat(theta_dot_dot, (U-(mc/2+ma) * g * l * cs.sin(theta)) / ((mc/3 + ma)* l**2 + I))
# Theta_dot_dot = cs.MX.sym("ttt")
# not sure
# X = cs.vertcat(theta)
X = cs.vertcat(theta)
Y = cs.vertcat(theta)

Q = cs.MX.sym('Q', nx, nx)
R = cs.MX.sym('R', nu, nu)

Xr = cs.MX.sym('Xr', nx, 1)
Ur = cs.MX.sym('Ur', nu, 1)

# convert angular to cartesian position 
x_x = l * cs.cos(theta)
x_z = l * cs.sin(theta) 

X_array = cs.vertcat(x_x, x_z)

cost_func = 0.5 * (X_array - Xr).T @ Q @ (X_array - Xr) + 0.5 * (U - Ur).T @ R @ (U - Ur)

dynamics = {"dyn_eqn": Theta_dot_dot, "obs_eqn": Y, "vars": {"X": X, "U": U}}
cost = {"cost_func": cost_func, "vars": {"X": X, "U": U, "Xr": Xr, "Ur": Ur, "Q": Q, "R": R}}
            

In [34]:
fc_func = cs.Function('fc', [x_x, U], [Theta_dot_dot], ['x', 'u'], ['f'])

RuntimeError: Error in Function::Function for 'fc' [MXFunction] at .../casadi/core/function.cpp:235:
.../casadi/core/function_internal.cpp:145: Error calling MXFunction::init for 'fc':
.../casadi/core/x_function.hpp:287: For fc: Xfunction input arguments must be purely symbolic.
Argument 0(x) is not symbolic.

In [19]:
from safe_control_gym.math_and_models.symbolic_systems import SymbolicModel
symbolic = SymbolicModel(dynamics=dynamics, cost=cost, dt=dt)

@1=vertcat(cos(theta), sin(theta)), (mac(mac((0.5*(@1-Xr)'),Q,zeros(1x2)),(@1-Xr),0)+(((0.5*(U-Ur))*R)*(U-Ur)))


RuntimeError: Error in Function::Function for 'fc' [MXFunction] at .../casadi/core/function.cpp:235:
.../casadi/core/function_internal.cpp:145: Error calling MXFunction::init for 'fc':
.../casadi/core/x_function.hpp:287: For fc: Xfunction input arguments must be purely symbolic.
Argument 0(x) is not symbolic.